In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Escaladores
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Train, Test
from sklearn.model_selection import train_test_split

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Métricas
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_to_encode.csv')

In [ ]:
df.head()

He hecho el onehot el knn el mapa de corr, feature importance, y buscar el mejor r2 falta que todo este en archivos pickel, porque queria que le dierais el visto bueno.

OneHotEncoder

In [ ]:
df_encoded = df.copy()

encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_features = encoder.fit_transform(df_encoded[['property_types']])
# Verifica el número de características generadas
feature_names = encoder.get_feature_names_out()
print(f'Number of feature names: {len(feature_names)}')
print(f'Number of encoded features: {encoded_features.shape[1]}')

# Crear el DataFrame con las características codificadas
encoded_df = pd.DataFrame(encoded_features, columns=feature_names)                                        

He cambiado los nombres para que sea mas fácil para nosotros.

In [ ]:
encoded_df = encoded_df.rename(columns={'property_types_Habitación': 'habitacion'})
encoded_df = encoded_df.rename(columns={'property_types_otro': 'alojamiento entero'})
encoded_df.head()

Lo uno todo a nuestro df principal y dropeo la que nos sobra también cambio a int64 (lo he tenido que poner asi porque se cambiaba a int32 cosa que no me habia pasado antes creo)

In [ ]:
df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
df_encoded.head()

In [ ]:
df_encoded = df_encoded.drop('property_types',axis= 1)

In [ ]:
df_encoded['habitacion'] = df_encoded['habitacion'].astype('int64')

In [ ]:
df_encoded['alojamiento entero'] = df_encoded['alojamiento entero'].astype('int64')

In [ ]:
df_encoded.info()

KNNImputer

In [ ]:
%%time
imputer = KNNImputer(n_neighbors=3)
df_encoded = pd.DataFrame(imputer.fit_transform(df_encoded), columns=df_encoded.columns)
df_encoded

Creamos una copia de nuestro df

In [ ]:
df = df_encoded.copy()

In [ ]:
df.head()

Heatmap

In [ ]:
plt.figure(figsize=(30, 20))

sns.heatmap(data = df_encoded.corr(), annot = True,cmap="Blues")

Feature Importance se observa que los servicios tienen algo que ver con los precios.

In [ ]:
%%time

# Feature Importance

# Modelo de RandomForest para obtener Feature Importance
model = RandomForestRegressor(n_estimators = 250,
                               random_state = 0)
# Entrenamos el modelo
model.fit(df.drop(["prices_per_night"], axis = 1), df["prices_per_night"])

# Calculamos Feature Importance
importances = model.feature_importances_

df = pd.DataFrame(data = zip(df.drop(["prices_per_night"], axis = 1).columns, importances),
                              columns = ["Columnas", "Importancia"])

df = df.sort_values("Importancia", ascending = False)

print("Feature Importance:")

for index, (feature, importance) in enumerate(df.values):
    
    print(f"{index + 1:2}. feature {index:2} ({importance:20}): {feature}")

plt.figure(figsize = (12, 8))

plt.title("Feature Importances")
sns.barplot(x = df["Importancia"], y = df["Columnas"], palette = sns.color_palette("husl", 17))

plt.grid()
plt.show()

TRAIN TEST

In [ ]:
X = df_encoded.drop("prices_per_night", axis = 1)
y = df_encoded["prices_per_night"]
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

Escalado

In [ ]:
x_scaler = MinMaxScaler()
X_train = x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)

y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(np.array(y_train).reshape(-1, 1))
y_test = y_scaler.transform(np.array(y_test).reshape(-1, 1))

Buscar el mejor modelo copn el r2 mas alto 

In [ ]:
models = {"Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Support Vector Regressor": SVR()}

In [ ]:
resultados_lista = []

for model_name, model in models.items():
    model.fit(X_train, y_train)
    
    y_hat = model.predict(X_test)
    
    y_test_inv = y_scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()
    y_hat_inv = y_scaler.inverse_transform(y_hat.reshape(-1, 1)).ravel()
    
    mae = mean_absolute_error(y_test_inv, y_hat_inv)
    mse = mean_squared_error(y_test_inv, y_hat_inv)
    r2 = r2_score(y_test_inv, y_hat_inv)
    
    resultados_lista.append({"model_name": model_name,"mae": mae,"mse": mse,"r2_score": r2})
resultados = pd.DataFrame(resultados_lista)
resultados = resultados.sort_values(by="r2_score", ascending=False)

In [ ]:
resultados